In [1]:
import random

from datasets import load_dataset

In [2]:
dataset = load_dataset("jerryjalapeno/nart-100k-synthetic")

In [3]:
print(dataset["train"].features)

{'id': Value(dtype='string', id=None), 'conversations': [{'from': Value(dtype='string', id=None), 'value': Value(dtype='string', id=None)}]}


In [5]:
def tranform_data(sample):
    conversation = []
    sample = sample["conversations"]
    prompt_message = """You are a helpful and joyous mental therapy assistant. Always answer as helpfully and cheerfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""
    system_prompt = f"<<SYS>>\n{prompt_message}\n<</SYS>\n\n"
    for i, s in enumerate(sample):
        if(s["from"]) == "human":
            # Replace any mention of Alex with empty string and remove any extra spaces after and , and . , due to removing the word.
            s["value"] = s["value"].replace("Alex", "")
            s["value"] = s["value"].replace("Charlie", "")
            s["value"] = s["value"].replace(" ,", ",")
            s["value"] = s["value"].replace(", .", ". ")
            if(i == 0):
                conversation.append(f"<s>[INST] {system_prompt}{s['value']} [/INST]")
            else:
                conversation.append(f"<s>[INST] {s['value']} [/INST]")
        else:
            s["value"] = s["value"].replace("Charlie", "")
            s["value"] = s["value"].replace("Alex", "")
            s["value"] = s["value"].replace(" ,", ",")
            s["value"] = s["value"].replace(", .", ". ")
            conversation.append(f" {s['value']} </s>")
    # generate a random integer between 0 and 10
    random_int = random.randint(0, 10)
    if(random_int == 1):
        # conversation list except the last element
        conv = "".join(conversation[:-1])
    else:
        conv = "".join(conversation)
    return {"text": conv}
# print(dataset["train"][0])
tranform_data(dataset["train"][0])

{'text': "<s>[INST] <<SYS>>\nYou are a helpful and joyous mental therapy assistant. Always answer as helpfully and cheerfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n<</SYS>\n\nI've been feeling so sad and overwhelmed lately. Work has become such a massive source of stress for me. [/INST] Hey there, I'm here to listen and support you. It sounds like work has been really challenging lately. Can you tell me more about what's been going on? </s><s>[INST] I recently got a promotion at work, which I thought would be exciting. But the added responsibilities and pressure have just taken a toll on my mental h

In [6]:
transformed_dataset = dataset.map(tranform_data,remove_columns=["conversations","id"])

In [7]:
print(transformed_dataset["train"].features)

{'text': Value(dtype='string', id=None)}


In [20]:
print(transformed_dataset)
print(transformed_dataset["train"][0])

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 99086
    })
})
{'text': "<s>[INST] <<SYS>>\nYou are a helpful and joyous mental therapy assistant. Always answer as helpfully and cheerfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n<</SYS>\n\nI've been feeling so sad and overwhelmed lately. Work has become such a massive source of stress for me. [/INST] Hey there, I'm here to listen and support you. It sounds like work has been really challenging lately. Can you tell me more about what's been going on? </s><s>[INST] I recently got a promotion at work, which I thought wou

In [21]:
transformed_dataset.push_to_hub("phr_mental_health_dataset")

Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [01:31<00:00, 91.62s/it]
